## Preparing the dataset


In [2]:
!pip install argilla

  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached rich-13.9.4-py3-none-any.whl (242 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13

In [5]:
from datasets import Dataset
import argilla as rg
import os
import uuid
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [22]:
settings = rg.Settings(
    guidelines="Annotated Universal Dependencies sentences with complexity metrics",
    allow_extra_metadata=True,
    fields=[
        rg.TextField(name="text", required=True),
        rg.TextField(name="question_id", required=True),
        rg.TextField(name="language", required=True),
        rg.TextField(name="n_tokens", required=True),
        rg.TextField(name="char_per_tok", required=True),
        rg.TextField(name="avg_max_depth", required=True),
        rg.TextField(name="lexical_density", required=True),
        rg.TextField(name="avg_links_len", required=True),
        rg.TextField(name="verbal_head_per_sent", required=True),
        rg.TextField(name="avg_prepositional_chain_len", required=True)
    ],
    questions = [
        rg.LabelQuestion(name="processed", title="processing status", description="shows if processed", required=True, labels=["ok","not ok"])
])

In [23]:
client = rg.Argilla(
    api_url="https://rokii3-complexity-annotated-questions.hf.space",
    api_key="CJ83an24doRX_To1xKivqZ0H6KLGBxDftdG0iSWcWtuTiP33VxpZ0z-d2ODQgKYQoc0DQFj_SskZD4jlumdsAiRi18YZ6sdK16p_XOTDl6M",
)

In [ ]:
FEATS = ['n_tokens','char_per_tok', 'lexical_density', 'avg_max_depth', 'avg_links_len', 'verbal_head_per_sent', 'avg_prepositional_chain_len']

In [ ]:
# Argilla boilerplate

In [28]:
dataset = rg.Dataset(
    name="question-data-linguistic-features",
    settings=settings,
)

dataset.create()


Dataset(id=UUID('09ded185-06b6-4c1b-9ec4-402bc01d77da') inserted_at=datetime.datetime(2025, 2, 22, 22, 6, 3, 826365) updated_at=datetime.datetime(2025, 2, 22, 22, 6, 6, 419074) name='question-data-linguistic-features' status='ready' guidelines='Annotated Universal Dependencies sentences with complexity metrics' allow_extra_metadata=True distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('581b13ef-9395-456e-9754-401d3b9abf28') last_activity_at=datetime.datetime(2025, 2, 22, 22, 6, 6, 419074))

In [43]:
import argilla as rg
import pandas as pd
from pathlib import Path
import uuid

def create_argilla_dataset(sample_record):
 
    fields = [
        rg.TextField(name="text", required=True),
        rg.TextField(name="question_id", required=True),
        rg.TextField(name="language", required=True)
    ]
    
    

    # Define settings
    settings = rg.Settings(
    guidelines="Annotated Universal Dependencies sentences with complexity metrics",
    allow_extra_metadata=True,
    fields=[
        rg.TextField(name="text", required=True),
        rg.TextField(name="question_id", required=True),
        rg.TextField(name="language", required=True),
        rg.TextField(name="n_tokens", required=True),
        rg.TextField(name="char_per_tok", required=True),
        rg.TextField(name="avg_max_depth", required=True),
        rg.TextField(name="lexical_density", required=True),
        rg.TextField(name="avg_links_len", required=True),
        rg.TextField(name="verbal_head_per_sent", required=True),
        rg.TextField(name="avg_prepositional_chain_len", required=True)
    ],
    questions = [
        rg.LabelQuestion(name="processed", title="processing status", description="shows if processed", required=True, labels=["ok","not ok"])
])


    dataset = rg.Dataset(name="ud-syntactic-features", settings=settings)
    return dataset

def process_file(file_path):
    language = Path(file_path).stem.split('-')[1]
    df = pd.read_csv(file_path)
    
    # Get text column (first column)
    text_col = df.columns[0]
    print(f"Text column name: {text_col}")

    records = []
    for _, row in df.iterrows():
        if pd.isna(row[text_col]):
            continue

    record = {
            "text": str(row[text_col]),
            "question_id": str(uuid.uuid4()),
            "language": language,
            "verification": "needs_review"
        }
    
    record['metadata'] = {}
    columns_to_skip = [text_col, 'Filename', 'base_number']
    # Add all features except metadata columns
    columns_to_skip = [text_col, 'Filename', 'base_number']
    for col in df.columns:
        if col not in columns_to_skip:
            try:
                value = row[col] if not pd.isna(row[col]) else 0.0
                record[col] = float(value)
            except (ValueError, TypeError):
                print(f"Warning: Could not convert value for {col}: {row[col]}")
                record[col] = 0.0
                    
        records.append(record)
    
    print(f"Created {len(records)} records from {len(df)} rows")
    return records
  

def main():
   
    
    input_dir = Path('/home/robin/Research/qtype-eval/data/annotated_UD_questions')
    files = list(input_dir.glob("*.csv"))
    
    
    
    # Create and initialize dataset
    try:
        dataset = create_argilla_dataset(all_records[0])
        dataset.create()  # Initialize the dataset
        print("\nSuccessfully created dataset")
    except Exception as e:
        print(f"\nError creating dataset: {str(e)}")
        return
    
    all_records = []
    for file_path in files:
        print(f"\nProcessing {file_path.name}...")
        records = process_file(file_path)
        all_records.extend(records)
        print(f"Extracted {len(records)} records")
    
    if not all_records:
        print("No records found!")
        return
    
    
    print(f"\nTotal records to upload: {len(all_records)}")
    print(all_records[0])
    
    # Upload in batches using records.log()
    batch_size = 100
    total_batches = (len(all_records) + batch_size - 1) // batch_size
    
    for i in range(0, len(all_records), batch_size):
        batch = all_records[i:i + batch_size]
        try:
            dataset.records.log(records=batch)  # Using records.log() instead of add_records()
            print(f"Uploaded batch {i//batch_size + 1}/{total_batches}")
        except Exception as e:
            print(f"Error uploading batch: {str(e)}")
            print("First record in failed batch:")
            print(batch[0])

if __name__ == "__main__":
    main()


Error creating dataset: cannot access local variable 'all_records' where it is not associated with a value


In [ ]:
def process_file(file_path):
  language = Path(file_path).stem.split('-')[1]

  df = pd.read_csv(file_path)

  text = df.columns[0]

  
  features = ['text'] + [f for f in FEATS if f in df.columns]
  df_filtered = df[features].copy()
    
  records = []
  for _, row in df_filtered.iterrows():
    if pd.isna(row[text]):
      continue

    record = {
            "text": str(row['text']),
            "question_id": str(uuid.uuid4()),
            "language": language,
            "verification": "needs_review" 
    }

    columns_to_skip = [text, 'Filename', 'base_number']
        
    for col in df.columns:
        if col not in columns_to_skip:
            try:
                value = row[col] if not pd.isna(row[col]) else 0.0
                record[col] = float(value)
            except (ValueError, TypeError):
                print(f"Warning: Could not convert value for {col}: {row[col]}")
                record[col] = 0.0
    
    records.append(record)
    
    print(f"Created {len(records)} records from {len(df)} rows")
    return records
    
    

def main():

  input_dir = Path('/home/robin/Research/qtype-eval/data/annotated_UD_questions')
  files = list(input_dir.glob("*.csv"))

  all_records = []
  for file_path in files:
    print(f"\nProcessing {file_path.name}...")
    records = process_file(file_path)
    all_records.extend(records)
    print(f"Extracted {len(records)} records")
    
  print(f"\nTotal records to upload: {len(all_records)}")

  batch_size = 100
  for i in range(0, len(all_records), batch_size):
    batch = all_records[i:i + batch_size]
    try:
      dataset.add_records(batch)
      print(f"Uploaded batch {i//batch_size + 1}/{(len(all_records) + batch_size - 1)//batch_size}")
    except Exception as e:
      print(f"Error uploading batch: {str(e)}")

if __name__ == "__main__":
  main()


Processing UD-korean-polar-annotated.csv...


KeyError: "['text'] not in index"

In [ ]:
display(df)